In [85]:
from datetime import datetime
from collections import Counter
import csv
# Data management
import pandas as pd
from itertools import combinations
# Data preprocessing and trasformation (ETL)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, FunctionTransformer, Binarizer, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml, load_iris, make_moons, make_classification
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Math and Stat modules
import numpy as np
from scipy.stats import sem
from random import choice

# Supervised Learning
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, StratifiedKFold, RepeatedKFold, ShuffleSplit, StratifiedShuffleSplit, learning_curve, validation_curve
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

# Unsupervised Learning

# Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

# Network
import networkx as nx

In [76]:
spotify_temporal = nx.Graph()

In [82]:
with open('01-global.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        s=row[3].replace("[","")
        s=s.replace("]","")
        s=s.replace("'","")
        artisti = s.split(",")
        if len(artisti) > 1:            
            # Calcoliamo tutte le possibili coppie di elementi
            coppie = list(combinations(artisti, 2))

            # Stampiamo le coppie ottenute
            for coppia in coppie:
                spotify_temporal.add_edge(coppia[0], coppia[1])

print(spotify_temporal)

Graph with 5 nodes and 4 edges


In [84]:
for i in range (2, 16):
    with open(f'0{i}-global.csv', 'r', encoding = 'utf-16') as f:
        csv_reader = csv.reader(f)
        for row in csv_reader:
            s=row[3].replace("[","")
            s=s.replace("]","")
            s=s.replace("'","")
            artisti = s.split(",")
            if len(artisti) > 1:            
                # Calcoliamo tutte le possibili coppie di elementi
                coppie = list(combinations(artisti, 2))

                # Stampiamo le coppie ottenute
                for coppia in coppie:
                    spotify_temporal.add_edge(coppia[0], coppia[1])

print(spotify_temporal.edges)

[('FloyyMenor', ' Cris Mj'), ('Future', ' Metro Boomin'), ('Future', ' Kendrick Lamar'), (' Metro Boomin', ' Kendrick Lamar'), ('The Weeknd', ' JENNIE'), ('The Weeknd', ' Lily-Rose Depp'), (' JENNIE', ' Lily-Rose Depp'), ('Taylor Swift', ' Post Malone'), ('Taylor Swift', ' Florence + The Machine')]


In [90]:
import networkx as nx

# Inserisci le tue credenziali Spotify qui
#client_id = '85aa8b30835e45a3b99623b2797df916'
#client_secret = 'ecb8aa6ad9d7431db74fb99af51dec93'

# Autenticazione con Spotify
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))


# Funzione per ottenere gli artisti il cui nome inizia con la lettera 'a'
def get_artists_starting_with_a():
    artists = set()
    for i in range(0, 10000, 50):  # Possiamo ottenere fino a 50 risultati per richiesta, quindi iteriamo per ottenere più risultati
        results = sp.search(q='artist', type='artist', limit=50, offset=i)
        for artist in results['artists']['items']:
            if artist['name'].lower().startswith('a'):
                artists.add(artist['id'])
    return list(artists)

# Funzione per ottenere le collaborazioni tra artisti
def get_collaborations(artists):
    collaborations = []
    for artist_id in artists:
        # Otteniamo le tracce in cui l'artista ha collaborato
        tracks = sp.artist_top_tracks(artist_id)
        for track in tracks['tracks']:
            # Per ogni traccia, otteniamo gli artisti che hanno partecipato
            track_artists = [artist['id'] for artist in track['artists'] if artist['id'] != artist_id]
            # Se ci sono più di 2 artisti, aggiungiamo tutte le possibili coppie
            if len(track_artists) >= 2:
                collaborations.extend([(track_artists[i], track_artists[j]) for i in range(len(track_artists)) for j in range(i+1, len(track_artists))])
    return collaborations

# Creiamo il grafo
def build_graph(collaborations):
    G = nx.Graph()
    G.add_edges_from(collaborations)
    return G

# Otteniamo gli artisti il cui nome inizia con 'a'
artists_starting_with_a = get_artists_starting_with_a()

# Otteniamo le collaborazioni tra gli artisti
collaborations = get_collaborations(artists_starting_with_a)

# Costruiamo il grafo
graph = build_graph(collaborations)

# Ora puoi fare operazioni sul grafo, ad esempio contare il numero di artisti collegati
num_nodes = len(graph.nodes())
print("Numero di artisti collegati:", num_nodes)

# O anche ottenere informazioni sui collegamenti
num_edges = len(graph.edges())
print("Numero di collaborazioni:", num_edges)


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist', 'limit': 50, 'offset': 1000, 'type': 'artist', 'market': None} returned 400 due to Bad request.


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=artist&limit=50&offset=1000&type=artist:
 Bad request., reason: None